In [2]:
import molvs
import pandas as pd
from rdkit.Chem import MolFromSmiles
from rdkit import Chem

In [3]:
data = pd.read_csv("filtered_pgp.csv")

In [4]:
data

,Unnamed: 0,Drugs,SMILES
0,0,Verapamil,COC1=C(OC)C=C(CCN(C)CCCC(C#N)(C(C)C)C2=CC(OC)=...
1,1,Saquinavir,[H][C@@]12CCCC[C@]1([H])CN(C[C@@H](O)[C@H](CC1...
2,2,Nelfinavir,[H][C@@]12CCCC[C@]1([H])CN(C[C@@H](O)[C@H](CSC...
3,3,Indinavir,CC(C)(C)NC(=O)[C@@H]1CN(CC2=CN=CC=C2)CCN1C[C@@...
4,4,Reserpine,[H][C@]12C[C@@H](OC(=O)C3=CC(OC)=C(OC)C(OC)=C3...
...,...,...,...
353,365,Riociguat,COC(=O)N(C)C1=C(N)N=C(N=C1N)C1=NN(CC2=C(F)C=CC...
354,366,Avacopan,CC1=CC=C(NC(=O)[C@H]2CCCN([C@H]2C2=CC=C(NC3CCC...
355,367,Asciminib,O[C@@H]1CCN(C1)C1=C(C=C(C=N1)C(=O)NC1=CC=C(OC(...
356,368,Nirmatrelvir,[H][C@]12CN([C@H](C(=O)N[C@@H](C[C@]3([H])CCNC...


In [5]:
data["Molecule"] = data.SMILES.apply(MolFromSmiles)

In [6]:
from molvs.standardize import Standardizer

In [7]:
s = Standardizer()
standarizedMol = []
for i in data.Molecule:
    s_mol = s.standardize(i)
    standarizedMol.append(s_mol)

In [8]:
standarizedMol

In [9]:
standarizedSmi = [Chem.MolToSmiles(i) for i in standarizedMol]

In [10]:
standarizedSmi

['COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C)cc1OC',
 'CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)c1ccc2ccccc2n1',
 'Cc1c(O)cccc1C(=O)N[C@@H](CSc1ccccc1)[C@H](O)CN1C[C@H]2CCCC[C@H]2C[C@H]1C(=O)NC(C)(C)C',
 'CC(C)(C)NC(=O)[C@@H]1CN(Cc2cccnc2)CCN1C[C@@H](O)C[C@@H](Cc1ccccc1)C(=O)N[C@H]1c2ccccc2C[C@H]1O',
 'COC(=O)[C@H]1[C@H]2C[C@@H]3c4[nH]c5cc(OC)ccc5c4CCN3C[C@H]2C[C@@H](OC(=O)c2cc(OC)c(OC)c(OC)c2)[C@@H]1OC',
 'COC(=O)C1=C(C)NC(C)=C(C(=O)OC)C1c1ccccc1[N+](=O)[O-]',
 'C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@]2(C)[C@@]1(O)C(=O)CO',
 'NC(=O)N1c2ccccc2C=Cc2ccccc21',
 'CC(C)c1nc(CN(C)C(=O)N[C@H](C(=O)N[C@@H](Cc2ccccc2)C[C@H](O)[C@H](Cc2ccccc2)NC(=O)OCc2cncs2)C(C)C)cs1',
 'C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[C@@H](O)C[C@H](O[C@H]4CC[C@]5(C)[C@H]6C[C@@H](O)[C@]7(C)[C@@H](C8=CC(=O)OC8)CC[C@]7(O)[C@@H]6CC[C@@H]5C4)O[C@@H]3C)O[C@@H]2C)C[C@H](O)[C@@H]1O',
 'C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC)C(=O

In [ ]:
from rdkit.Chem import SaltRemover

In [12]:
querydata = pd.read_csv("FinalClearData.csv")

In [16]:
with open("querysmiles.smi","w") as file:
    for i in querydata.SMILES.tolist():
        file.write(i+"\n")

In [11]:
with open("Standardized.smi","w") as file:
    for i in standarizedSmi:
        file.write(i+"\n")

In [22]:
def converter(i,j):
    os.system(fr"""obabel -:"{i}" -ismi -osdf -O {j}_out.sdf -d -h --gen3D --minimize --steps 5000 --sd --ff MMFF94""",)
    return j

In [20]:
import concurrent
import os

In [21]:
SDFarray = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    out = {executor.submit(converter,i,idx): (i,idx) for idx,i in enumerate(standarizedSmi)}
    for future in tqdm(concurrent.futures.as_completed(out)):
        fp = future.result()
        print(fp)
        SDFarray.append(fp)  

*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 4 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 3 4 13 15
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 1 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 1
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 5 23
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 3 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 7 12 20
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 14 27
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 4 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 20 22 26 42
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 1 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 5
11 molecule converted
 molecule

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 1 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 10
1 molecule converted
1 molecule converted
*** Open Babel Warning  in WriteMolecule
  No 2D or 3D coordinates exist. Stereochemical information will be stored using an Open Babel extension. To generate 2D or 3D coordinates instead use --gen2D or --gen3D.
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  w

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 4 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 11 12 21 23
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in WriteMolecule
  No 2D or 3D coordinates exist. Stereochemical information will be stored using an Open Babel extension. To generate 2D or 3D coordinates instead use --gen2D or --gen3D.
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule con

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
Process ForkProcess-25:
Process ForkProcess-5:
Process ForkProcess-4:
Process ForkProcess-20:
Process ForkProcess-10:
Process ForkProcess-27:
Process ForkProcess-16:
Process ForkProcess-23:
Process ForkProcess-11:
Process ForkProcess-32:
Process ForkProcess-17:
Process ForkProcess-2:
Process ForkProcess-21:
Process ForkProcess-7:
Process ForkProcess-28:
Process ForkProcess-9:
Process ForkProcess-3:
Process ForkProcess-8:
Process ForkProcess-26:
Process ForkProcess-24:
Process ForkProcess-12:
Process ForkProcess-15:
Process ForkProcess-1:
Process ForkProcess-19:
Process ForkProcess-31:
Process ForkProcess-18:
Process ForkProcess-30:
Process ForkProcess-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocess

  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/queues.py", line 102, in get
    with self._rlock:
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/concurr

  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/saurav/anaconda3/envs/prolif/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
Keyb

In [ ]:
for idx,i in enumerate(standarizedSmi):
    s = converter(i,idx)
    print(s)
    

1 molecule converted


0


*** Open Babel Error  in Do
  3D coordinate generation failed
*** Open Babel Warning  in WriteMolecule
  No 2D or 3D coordinates exist. Stereochemical information will be stored using an Open Babel extension. To generate 2D or 3D coordinates instead use --gen2D or --gen3D.
1 molecule converted


1


*** Open Babel Error  in Do
  3D coordinate generation failed
*** Open Babel Warning  in WriteMolecule
  No 2D or 3D coordinates exist. Stereochemical information will be stored using an Open Babel extension. To generate 2D or 3D coordinates instead use --gen2D or --gen3D.
1 molecule converted


...

1 molecule converted


35


1 molecule converted
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 8 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 6 12 14 55 59 79 97 112


36
37


*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 3 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 1 5 6
